In [1]:
import requests

auth_url = "https://signin.tradestation.com/authorize"

public_key = "r4bJ08Nbz9f8b6djDhoyCmazNnrrLFL4"
private_key = "hFBk8xgV_UGJEUFnxVW-AFz6YToqZwdvM-48x5wLUQhzKiR99r2w780hL0giBfvd"

params = {"response_type": "code", "client_id": public_key, "redirect_uri": "http://localhost", "audience": "https://api.tradestation.com", "state": "STATE", 
        "scope": "openid offline_access profile MarketData ReadAccount Trade Crypto Matrix OptionSpreads"}

response = requests.get(auth_url, params=params)
response.history
for resp in response.history:
        print(resp.status_code, resp.url)

302 https://signin.tradestation.com/authorize?response_type=code&client_id=r4bJ08Nbz9f8b6djDhoyCmazNnrrLFL4&redirect_uri=http%3A%2F%2Flocalhost&audience=https%3A%2F%2Fapi.tradestation.com&state=STATE&scope=openid+offline_access+profile+MarketData+ReadAccount+Trade+Crypto+Matrix+OptionSpreads


In [ ]:
import requests

token_url = "https://signin.tradestation.com/oauth/token"

authorization_code = "xxMZvFaqtX95pCIj"

headers = {"content-type": "application/x-www-form-urlencoded"}
data = {"grant_type": "authorization_code", "client_id": public_key, "client_secret": private_key, "code": authorization_code, "redirect_uri": "http://localhost"}

r = requests.post(token_url, headers=headers, data=data).json()
print(r)

access_token = r["access_token"]
refresh_token = r["refresh_token"]
id_token = r["id_token"]

In [2]:
import requests
refresh_token = "lzijZJe0VUnjFEzq6dqWK-Q3rqhzyoq7kkrDkqVoJvoYn"
def refresh():
    token_url = "https://signin.tradestation.com/oauth/token"

    headers = {"content-type": "application/x-www-form-urlencoded"}
    data = {"grant_type": "refresh_token", "client_id": public_key, "client_secret": private_key, "refresh_token": refresh_token}

    r = requests.post(token_url, headers=headers, data=data).json()

    access_token = r["access_token"]
    return access_token

In [ ]:
import requests
import pandas as pd
import json
import csv

url = "https://api.tradestation.com/v3/marketdata/barcharts/TSLA"

headers = {"Authorization": f"Bearer {access_token}"}

params = {"unit": "Minute", "firstdate": "2016-12-05T21:00:00Z", "lastdate": "2016-12-06T21:00:00Z"}

response = requests.request("GET", url, headers=headers, stream=True, params=params)
print(response)
if response.status_code == 401:
    access_token = refresh()
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.request("GET", url, headers=headers, stream=True, params=params)
    print(response)
response = response.json()
    
fieldnames = list(response["Bars"][0].keys())
with open("tradestation_data/tsla_2016_12_05_2016_12_06.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in response["Bars"]:
        writer.writerow(row)

In [ ]:
def stream_lvl1_quotes(symbol):
    url = f"https://api.tradestation.com/v3/marketdata/stream/quotes/{symbol}"
    while True:
        try:
            access_token = refresh()
            headers = {"Authorization": f"Bearer {access_token}"}

            response = requests.get(url, headers=headers, stream=True)
            if response.status_code != 200:
                print(f"{symbol} {response} {response.text}")
                continue   

            for line in response.iter_lines():
                if line:
                    row = json.loads(line)
                    if "Error" in row:
                        print(f"{symbol} {response} {response.text}")
                        continue
                    if "Heartbeat" in row:
                        continue
                    print(row)
        except Exception as e:
            print(f"{symbol} {e}")
            continue

In [ ]:
import os
import csv
import json
import pandas as pd
import datetime
import time

def stream_lvl2_quotes(symbol):
    url = f"https://api.tradestation.com/v3/marketdata/stream/marketdepth/aggregates/{symbol}"
    while True:
        try:
            access_token = refresh()
            headers = {"Authorization": f"Bearer {access_token}"}

            response = requests.get(url, headers=headers, stream=True)
            if response.status_code != 200:
                print(f"{symbol} {response} {response.text}")
                continue            
            
            for line in response.iter_lines():
                if line:
                    bids_and_asks = json.loads(line)
                    if "Heartbeat" in bids_and_asks:
                        continue
                    print(bids_and_asks)
        except Exception as e:
            print(f"{symbol} {e}")
            break
                    
symbols = "TSLA"
stream_lvl2_quotes(symbols)

In [1]:
from test_mongo import MongoExplorer
mongo = MongoExplorer()
mongo

ModuleNotFoundError: No module named 'pymongo'

In [110]:
import numpy as np
import pandas as pd
def calc_gain(symbol, date, price_dif_threshold, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    start = f'{date}T13:30:00Z'
    end = f'{date}T20:00:00Z'
    query = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query)))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query)))
    df = pd.merge(df_1, df_2, on="CurTime")
    df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
    df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
    df.loc[df["Side"] == "Ask", "TotalSize"] *= -1
    df = df.loc[df["Dif"] >= price_dif_threshold]
    if len(df) < 2:
        return 0
    #df.to_csv(f"tradestation_data/tsla_{date}_df.csv")
    sum_df = df.groupby("CurTime")["TotalSize"].sum()
    #print(sum_df)
    final_df = pd.merge(sum_df, df_1, on="CurTime")[["TradeTime", "Last", "Ask", "Bid", "TotalSize"]]
    final_df = final_df.rename(columns={"TotalSize": "VolumeSum"})
    final_df["Sign"] = final_df["VolumeSum"] >= size_threshold
    final_df["Sign Change"] = final_df["Sign"] - final_df["Sign"].shift()
    if final_df.loc[0, "Sign"]:
        final_df.loc[0, "Sign Change"] = 1 
    if final_df.loc[len(final_df) - 2, "Sign"]:
        final_df.loc[len(final_df) - 1, "Sign Change"] = -1
    else:
        final_df.loc[len(final_df) - 1, "Sign Change"] = 0
    final_df.loc[final_df["Sign Change"] == 1, "Balance Change"] = -final_df.loc[final_df["Sign Change"] == 1, "Ask"]
    final_df.loc[final_df["Sign Change"] == -1, "Balance Change"] = final_df.loc[final_df["Sign Change"] == -1, "Bid"]
    
    final_df.loc[final_df["Sign Change"] == 1, "Short Balance Change"] = -final_df.loc[final_df["Sign Change"] == 1, "Ask"]
    final_df.loc[final_df["Sign Change"] == -1, "Short Balance Change"] = final_df.loc[final_df["Sign Change"] == -1, "Bid"]
    final_df["Rounded Last"] = (final_df["Last"] * 100).apply(np.floor) / 100
    final_df.loc[final_df["Sign Change"] == 1, "Last Balance Change"] = -final_df.loc[final_df["Sign Change"] == 1, "Rounded Last"]
    final_df.loc[final_df["Sign Change"] == -1, "Last Balance Change"] = final_df.loc[final_df["Sign Change"] == -1, "Rounded Last"]
    # final_df["LastDif"] = final_df["Last"].diff().shift(-1)
    # final_df["TrueDif"] = final_df["Bid"].shift(-1) - final_df["Ask"]
    # #final_df["SameSign"] = (final_df["VolumeSum"] > 0) == (final_df["LastDif"] > 0)
    # # print(final_df["SameSign"].sum())
    # # print(len(final_df))
    # final_df.loc[final_df["VolumeSum"]>=size_threshold, "Gain"] = final_df["LastDif"]
    # final_df.loc[final_df["VolumeSum"]<size_threshold, "Gain"] = -final_df["LastDif"]
    # final_df["Gain"] = (final_df["Gain"] * 100).apply(np.floor) / 100
    # final_df.loc[final_df["VolumeSum"]>=size_threshold, "TrueGain"] = final_df["TrueDif"]
    # final_df.loc[final_df["VolumeSum"]<size_threshold, "TrueGain"] = 0
    # final_df["TrueGain"] = (final_df["TrueGain"] * 100).apply(np.floor) / 100
    # print(final_df)
    final_df.to_csv(f"tradestation_data/{symbol}_{date}_gain.csv")
    #print(final_df)
    return final_df["Balance Change"].sum()
symbols = ['NVDA', 'AMC', 'META', 'NFLX', 'AMZN', 'AAPL', 'ZM', 'GOOG', 'MSFT', 'TSLA'] 
price_dif_threshold = .025
size_threshold = 0
# for price_dif_threshold in np.arange(.01, 0, -.0005):
# for size_threshold in range(0, 50, 5):
for symbol in symbols:
    for day in range(5, 9):
        date = f"2023-06-0{day}"
        print(f"{symbol} {date} {calc_gain(symbol, date, price_dif_threshold, size_threshold)}")


NVDA 2023-06-05 -0.18999999999994088
NVDA 2023-06-06 5.790000000000987
NVDA 2023-06-07 -11.140000000000043
NVDA 2023-06-08 13.609999999999843
AMC 2023-06-05 0
AMC 2023-06-06 0
AMC 2023-06-07 0
AMC 2023-06-08 0
META 2023-06-05 1.7400000000004638
META 2023-06-06 6.529999999999632
META 2023-06-07 0.9799999999999613
META 2023-06-08 5.6299999999999955
NFLX 2023-06-05 -6.060000000000173
NFLX 2023-06-06 -11.42000000000013
NFLX 2023-06-07 -7.829999999999359
NFLX 2023-06-08 0.24999999999965894
AMZN 2023-06-05 1.2499999999999858
AMZN 2023-06-06 1.8999999999999488
AMZN 2023-06-07 -3.1800000000000637
AMZN 2023-06-08 -0.06999999999995055
AAPL 2023-06-05 3.3199999999998795
AAPL 2023-06-06 1.9799999999999898
AAPL 2023-06-07 2.6599999999999966
AAPL 2023-06-08 -0.46999999999997044
ZM 2023-06-05 1.0199999999999818
ZM 2023-06-06 0.8699999999999903
ZM 2023-06-07 -3.1499999999999346
ZM 2023-06-08 0.4600000000000364
GOOG 2023-06-05 2.3400000000000176
GOOG 2023-06-06 1.4299999999999926
GOOG 2023-06-07 -1.690

In [45]:
def read_lvl1(symbol, time):
    collection = f"{symbol}_10sec_ts_lvl1"
    query = {"CurTime": time}
    lvl1 = None
    while lvl1 is None:
        lvl1 = mongo.mongoDB[collection].find_one(query)
        print(lvl1)
    return None
symbol = "TSLA"
time = "2023-06-07T18:56:40Z"
print(read_lvl1(symbol, time))

{'_id': ObjectId('6480d2e8323df6bf8bfc3f93'), 'CurTime': '2023-06-07T18:56:40Z', 'GetTime': '2023-06-07T18:56:39Z', 'Symbol': 'TSLA', 'TradeTime': '2023-06-07T18:56:39Z', 'Last': 225.1, 'Ask': 225.12, 'AskSize': 300.0, 'Bid': 225.1, 'BidSize': 200.0, 'Volume': 157711317.0, 'VWAP': 226.886874080586, 'Open': 228.0, 'High': 230.82961, 'Low': 223.2, 'PreviousClose': 221.31, 'PreviousVolume': 146911576.0, 'NetChange': 3.79, 'NetChangePct': 0.0171}
None


In [153]:
import datetime
timenow = datetime.datetime.utcnow()
print(timenow)
timenow = timenow.replace(second=timenow.second - timenow.second % 10)
print(timenow)

2023-06-07 19:10:33.328367
2023-06-07 19:10:30.328367


In [5]:
import requests
account_id = "11655345"
def buy_share(symbol):
    access_token = refresh()

    url = "https://api.tradestation.com/v3/orderexecution/orders"

    payload = {
        "AccountID": "11655345",
        "Symbol": f"{symbol}",
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "BUY",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {
        "content-type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)
buy_share("TSLA")

{"Orders":[{"Message":"Sent order: Buy 1 TSLA @ Market","OrderID":"967793406"}]}



In [11]:
import requests
def sell_share(symbol):
    access_token = refresh()

    url = "https://api.tradestation.com/v3/orderexecution/orders"

    payload = {
        "AccountID": "11655345",
        "Symbol": f"{symbol}",
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "SELL",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {
        "content-type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)
sell_share("TSLA")

{"Orders":[{"Message":"Sent order: Sell 1 TSLA @ Market","OrderID":"967802392"}]}



In [13]:
import requests
def check_status():
    access_token = refresh()
    url = "https://api.tradestation.com/v3/brokerage/accounts/11655345/orders"

    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.request("GET", url, headers=headers)

    print(response.text)
check_status()

{"Orders":[{"AccountID":"11655345","CommissionFee":"0","ClosedDateTime":"2023-06-07T19:53:10Z","Currency":"USD","Duration":"DAY","FilledPrice":"0","Legs":[{"QuantityOrdered":"1","ExecQuantity":"0","QuantityRemaining":"0","BuyOrSell":"Buy","Symbol":"TSLA","AssetType":"STOCK"}],"OrderID":"967805697","OpenedDateTime":"2023-06-07T19:53:10Z","OrderType":"Market","PriceUsedForBuyingPower":"224.98","RejectReason":"EC501: Day trading margin rules, this trade can possibly designate you as a pattern day trader, please call the Trade Desk at 800-871-3563 for more information.","Routing":"Intelligent","Status":"REJ","StatusDescription":"Rejected","UnbundledRouteFee":"0"},{"AccountID":"11655345","CommissionFee":"0","ClosedDateTime":"2023-06-07T19:53:00Z","Currency":"USD","Duration":"DAY","FilledPrice":"224.9699","Legs":[{"OpenOrClose":"Open","QuantityOrdered":"1","ExecQuantity":"1","QuantityRemaining":"0","BuyOrSell":"Buy","Symbol":"TSLA","AssetType":"STOCK","ExecutionPrice":"224.9699"}],"OrderID":

In [44]:
import datetime
import requests
account_id = "11655345"
def store_price(symbol):
    url = "https://api.tradestation.com/v3/orderexecution/orderconfirm"
    access_token = refresh()
    payload = {
        "AccountID": account_id,
        "Symbol": symbol,
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "BUY",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {"content-type": "application/json", "Authorization": f"Bearer {access_token}"}

    order_time = datetime.datetime.utcnow()
    time = order_time.replace(second=order_time.second - order_time.second%10).strftime("%Y-%m-%dT%H:%M:%SZ")
    response = requests.request("POST", url, json=payload, headers=headers).json()
    print(response)
    print(response["Confirmations"][0])
    buy_price = response["Confirmations"][0]["EstimatedPrice"]
    print(buy_price)
store_price("TSLA")

{'Confirmations': [{'OrderAssetCategory': 'EQUITY', 'Currency': 'USD', 'Route': 'Intelligent', 'TimeInForce': {'Duration': 'DAY'}, 'AccountID': '11655345', 'OrderConfirmID': 'WJ4EfcDjhUOE5ta3RebvZQ', 'EstimatedPrice': '226.53', 'EstimatedCost': '226.53', 'DebitCreditEstimatedCost': '226.53', 'EstimatedCommission': '0', 'SummaryMessage': 'Buy 1 TSLA @ Market'}]}
{'OrderAssetCategory': 'EQUITY', 'Currency': 'USD', 'Route': 'Intelligent', 'TimeInForce': {'Duration': 'DAY'}, 'AccountID': '11655345', 'OrderConfirmID': 'WJ4EfcDjhUOE5ta3RebvZQ', 'EstimatedPrice': '226.53', 'EstimatedCost': '226.53', 'DebitCreditEstimatedCost': '226.53', 'EstimatedCommission': '0', 'SummaryMessage': 'Buy 1 TSLA @ Market'}
226.53


In [1]:
import numpy as np
import pandas as pd
def calc_real_gain(symbol, date, price_dif_threshold, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    collection_3 = f"{symbol}_ts_trade_prices"
    start = f'{date}T14:46:30Z'
    end = f'{date}T20:00:00Z'
    query_1 = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query_1)))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query_1)))
    df = pd.merge(df_1, df_2, on="CurTime")
    df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
    df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
    df.loc[df["Side"] == "Ask", "TotalSize"] *= -1
    df = df.loc[df["Dif"] >= price_dif_threshold]
    
    df["VolumeSum"] = df["TotalSize"] * df["Dif"]
    df = df.rename(columns={"CurTime": "Time"})
    sum_df = pd.DataFrame(df.groupby("Time")["VolumeSum"].sum()).reset_index()
    #sum_df = sum_df.rename(columns={"TotalSize": "VolumeSum"})
    #print(sum_df)
    sum_df = sum_df[(sum_df["VolumeSum"] >= size_threshold) | (sum_df["VolumeSum"] <= -size_threshold)]
    sum_df["Long"] = sum_df["VolumeSum"] >= size_threshold
    sum_df["Short"] = sum_df["VolumeSum"] <= -size_threshold
    #print(sum_df)
    sum_df["Buy Action"] = sum_df["Long"] & sum_df["Short"].shift()
    if len(sum_df) < 3:
        return 0
    if sum_df.iloc[0, sum_df.columns.get_loc("Long")]:
        sum_df.iloc[0, sum_df.columns.get_loc("Buy Action")] = True
    sum_df.iloc[len(sum_df) - 1, sum_df.columns.get_loc("Buy Action")] = False
    sum_df["Sell Action"] = sum_df["Short"] & sum_df["Long"].shift()
    if sum_df.iloc[len(sum_df) - 2, sum_df.columns.get_loc("Long")] :
        sum_df.iloc[len(sum_df) - 1, sum_df.columns.get_loc("Sell Action")] = True

    query_3 = {'$and': [{'Time': {'$gte': start}}, {'Time': {'$lt': end}}]}
    df_3 = pd.DataFrame(list(mongo.mongoDB[collection_3].find(query_3)))

    final_df = pd.merge(sum_df, df_3, on="Time", how="right")
    final_df = final_df[["Time", "VolumeSum", "Long", "Short", "Buy Action", "Sell Action", "Last", "BUY", "SELL"]]
    final_df = final_df.fillna(False)
    final_df.loc[final_df["Buy Action"], "Gain"] = -final_df.loc[final_df["Buy Action"], "BUY"] 
    final_df.loc[final_df["Sell Action"], "Gain"] = final_df.loc[final_df["Sell Action"], "SELL"]
    #print(final_df)
    ret_list = [final_df["Gain"].sum()]
    # for i in range(1, 24, 1):
    #     final_df[f"{i}0sec Dif"] = final_df["SELL"].shift(-i) - final_df["BUY"]
    #     final_df[f"{i}0sec Short Dif"] = final_df["SELL"] - final_df["BUY"].shift(-i)
    #     final_df.loc[final_df["Long"], f"{i}0sec Gain"] = final_df[f"{i}0sec Dif"]
    #     final_df.loc[final_df["Short"], f"{i}0sec Short Gain"] = final_df[f"{i}0sec Short Dif"]
    #     ret_list.append([final_df[f"{i}0sec Gain"].sum(), final_df[f"{i}0sec Short Gain"].sum()])
    # final_df = final_df[final_df["VolumeSum"].notnull()]
    final_df.to_csv(f"tradestation_data/{symbol}_{date}_real_gain.csv")
    return ret_list
symbol = "TSLA"
price_dif_threshold = .1
size_threshold = 1
date = "2023-06-09"
# for size_threshold in range(1, 100, 1):
# log_list = []
# for price_dif_threshold in np.arange(.2, 0, -.001):
print([price_dif_threshold, size_threshold, calc_real_gain(symbol, date, price_dif_threshold, size_threshold)])
# out_df = pd.DataFrame(log_list)
# out_df.to_csv(f"tradestation_data/TSLA_{date}_threshold_tests.csv")


NameError: name 'mongo' is not defined

In [125]:
def get_mongo(symbol, date):
    collection = f"{symbol}_10sec_ts_lvl1"
    start = f'{date}T13:30:00Z'
    end = f'{date}T20:00:00Z'
    query = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df = pd.DataFrame(list(mongo.mongoDB[collection].find(query)))
    df = df[["CurTime", "Last", "Ask", "Bid"]]
    df.to_csv(f"tradestation_data/{symbol}_{date}_lvl1.csv")

symbol = "TSLA"
date = "2023-06-09"
get_mongo(symbol, date)

In [ ]:
import requests

access_token = refresh()
account_id = "11655345"
url = f"https://api.tradestation.com/v3/brokerage/stream/accounts/{account_id}/orders"

headers = {"Authorization": f"Bearer {access_token}"}

response = requests.request("GET", url, headers=headers, stream=True)

for line in response.iter_lines():
    if line:
        print(line)